In [12]:
import json
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

In [17]:
apiKey = "Q1HT11OLYGXHE6I5"
interval = 'daily'
period = '14'
series_type = 'close'
stocks = ['SPY', 'AAPL', 'AMZN', 'AXP', 'BAC', 'BK', 'JPM', 'KO', 'MCO', 'USB', 'WFC']

In [18]:
for stock in stocks:
    ticker = stock
    print(f'Collecting data for {ticker}....')
    
    # Daily Prices & Volumes
    stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey={apiKey}&outputsize=full"
    stock_response = requests.get(stock_url).json()
    print(f'Checking {ticker} price...')
    time.sleep(15)

    # SMA - Simple Moving Average
    sma_url = f"https://www.alphavantage.co/query?function=SMA&symbol={ticker}&interval={interval}&time_period={period}&series_type={series_type}&apikey={apiKey}"
    sma_response = requests.get(sma_url).json()
    print(f'Checking {ticker} SMA...')
    time.sleep(15)

    # RSI - Relative Strength Index
    rsi_url = f"https://www.alphavantage.co/query?function=RSI&symbol={ticker}&interval={interval}&time_period={period}&series_type={series_type}&apikey={apiKey}"
    rsi_response = requests.get(rsi_url).json()
    print(f'Checking {ticker} RSI...')
    time.sleep(15)

    # STOCH - Stochasitc Oscillator
    stoch_url = f"https://www.alphavantage.co/query?function=STOCH&symbol={ticker}&interval={interval}&apikey={apiKey}"
    stoch_response = requests.get(stoch_url).json()
    print(f'Checking {ticker} STOCH...')
    time.sleep(15)

    # ROC - Rate of Change
    roc_url = f"https://www.alphavantage.co/query?function=ROC&symbol={ticker}&interval={interval}&time_period={period}&series_type={series_type}&apikey={apiKey}"
    roc_response = requests.get(roc_url).json()
    print(f'Checking {ticker} ROC...')
    time.sleep(15)

    # WILLR - Williams %R
    willr_url = f"https://www.alphavantage.co/query?function=WILLR&symbol={ticker}&interval={interval}&time_period={period}&apikey={apiKey}"
    willr_response = requests.get(willr_url).json()
    print(f'Checking {ticker} WILLR...')
    time.sleep(15)

    # BBANDS - BOLLINGER BANDS
    bbands_url = f"https://www.alphavantage.co/query?function=BBANDS&symbol={ticker}&interval={interval}&time_period={period}&series_type={series_type}&nbdevup=3&nbdevdn=2&apikey={apiKey}"
    bbands_response = requests.get(bbands_url).json()
    print(f'Checking {ticker} BBANDS...')
    time.sleep(15)

    date = datetime(2010, 7, 8)
    dates = []
    open_amt = []
    high = []
    low = []
    close = []
    volume = []
    dividend = []
    split = []
    adj_close = []
    sma = []
    rsi = []
    slowd = []
    slowk = []
    roc = []
    willr = []
    std = []
    next_close = []

    daily_ticks = len(stock_response['Time Series (Daily)'])
    for x in range(daily_ticks):
        date += timedelta(days=1)
        new_date = date.strftime("%Y-%m-%d")
        try:
            stock_response['Time Series (Daily)'][new_date]
            try:
                dates.append(new_date)
            except: 
                dates.append('')
            try:
                open_amt.append(stock_response['Time Series (Daily)'][new_date]['1. open'])
            except: 
                open_amt.append('')
            try:
                high.append(stock_response['Time Series (Daily)'][new_date]['2. high'])
            except: 
                high.append('')
            try:
                low.append(stock_response['Time Series (Daily)'][new_date]['3. low'])
            except: 
                low.append('')
            try:
                close.append(stock_response['Time Series (Daily)'][new_date]['4. close'])
            except: 
                close.append('')
            try:
                volume.append(stock_response['Time Series (Daily)'][new_date]['6. volume'])
            except: 
                volume.append('')
            try:
                dividend.append(stock_response['Time Series (Daily)'][new_date]['7. dividend amount'])
            except: 
                dividend.append('')
            try:
                split.append(stock_response['Time Series (Daily)'][new_date]['8. split coefficient'])
            except: 
                split.append('')
            try:
                adj_close.append(stock_response['Time Series (Daily)'][new_date]['5. adjusted close'])
            except: 
                adj_close.append('')
            try:
                sma.append(sma_response['Technical Analysis: SMA'][new_date]['SMA'])
            except: 
                sma.append('')
            try:
                rsi.append(rsi_response['Technical Analysis: RSI'][new_date]['RSI'])
            except: 
                rsi.append('')
            try:
                slowd.append(stoch_response['Technical Analysis: STOCH'][new_date]['SlowD'])
            except: 
                slowd.append('')
            try:
                slowk.append(stoch_response['Technical Analysis: STOCH'][new_date]['SlowK'])
            except: 
                slowk.append('')
            try:
                roc.append(roc_response['Technical Analysis: ROC'][new_date]['ROC'])
            except: 
                roc.append('')
            try:
                willr.append(willr_response['Technical Analysis: WILLR'][new_date]['WILLR'])
            except: 
                willr.append('')
            try:
                std.append(str((float(bbands_response['Technical Analysis: BBANDS'][new_date]['Real Upper Band']) - float(bbands_response['Technical Analysis: BBANDS'][new_date]['Real Middle Band'])) / 2))
            except: 
                std.append('')
        except:
            print("skipped date")

    next_close = close.copy()
    next_close.pop(0)
    next_close.append('')

    df = pd.DataFrame({
        'ticker': ticker,
        'date': dates,
        'open': open_amt,
        'high': high,
        'low': low,
        'close': close,
        'volume': volume,
        'dividend': dividend,
        'split': split,
        'adj_close': adj_close,
        'sma': sma,
        'rsi': rsi,
        'slowd': slowd,
        'slowk': slowk,
        'roc': roc,
        'willr': willr,
        'std': std,
        'next_close': next_close
    })

    df['direction'] = ''

    for index, row in df.iterrows():
        if row['next_close'] != '':
            change = float(row['next_close']) - float(row['close'])
            if change > 0:
                row['direction'] = '1'
            elif change == 0:
                row['direction'] = '0'
            else:
                row['direction'] = '-1'

    df.to_csv(f"data/{ticker}.csv", index=False, header=True)

Checking KO price...
Checking KO SMA...
Checking KO RSI...
Checking KO STOCH...
Checking KO ROC...
Checking KO WILLR...
Checking KO BBANDS...
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date


skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date

Checking MCO price...
Checking MCO SMA...
Checking MCO RSI...
Checking MCO STOCH...
Checking MCO ROC...
Checking MCO WILLR...
Checking MCO BBANDS...
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skippe

skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date

skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date
skipped date